**Group 007-13 Project Proposal**

**Introduction**
The data set we chose is a tennis data set. We chose to keep specific columns in order to remove irrelevant data and keep the most important columns. We decided to keep if they are right or left handed, their current ranking, the player's name, their backhand throw, their current elo rank, and their prize money. A player's current elo rank is an estimate of their strength and is affected by their wins and loses. We are wondering whether a players bankhand throw and whether they are left or right handed affects their overall success. We will evaluate this by using their backhand, handedness, current rank, and current elo rank. 

**Preliminary exploratory data analysis**
*In code cells below, demonstrate that data can be read into R and then follow the instructions on Canvas*

In [35]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

download.file("https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn", "tennis_data.csv")
tennis_data <- read_csv("tennis_data.csv", show_col_types = FALSE) |> 
    mutate(play_surface = as_factor(surface), winner_hand = as_factor(winner_hand), loser_hand = as_factor(loser_hand)) |>
    mutate(total_age = winner_age + loser_age, total_rank_points = winner_rank_points + loser_rank_points, total_height = winner_ht + loser_ht) |>
    select(play_surface, total_rank_points, total_age, total_height, winner_hand, winner_age, winner_ht, loser_hand, loser_age, winner_ht, minutes, winner_rank_points, loser_rank_points)
#tennis_data

tennis_split <- initial_split(tennis_data, prop = .75, strata = )  
tennis_train <- training(tennis_split)   
tennis_test <- testing(tennis_split)
# tennis_train
# tennis_test

demo_plot <- ggplot(tennis_test, aes(x = total_rank_points, y = total_age, colour = play_surface)) +
    geom_point() #+
    #labs(title = "Players' Ages and Playing Surface", x = "Winner's Age", y = "Play Time (minutes)", colour = "Playing Surface")
demo_plot

ERROR: Error in parse(text = x, srcfile = src): <text>:10:139: unexpected symbol
9: oser_age, total_rank_points = winner_rank_points + loser_rank_points, total_height = winner_ht + loser_ht) |>
10:     select(play_surface, total_rank_points, total_age, total_height, winner_hand, winner_age, winner_ht, loser_hand, loser_age, winner_ht minutes
                                                                                                                                              ^


**Methods**
*Explain how you will conduct either your data analysis and which variables/columns you will use.
Describe at least one way that you will visualize the results*

**Expected outcomes and significance** *Answer questions on Canvas*